Wahlpflichtfach Künstliche Intelligenz II: Praktikum 

---

# 01 - Natural Language Processing (NLP)

## Daten importieren und analysieren

Im Folgenden wird ein [imdb](https://huggingface.co/datasets/imdb) Datensatz mit Reviews verwendet, um herauszufinden ob dieses Positiv oder negativ war.

In [ ]:
from datasets import load_dataset

dataset = load_dataset("imdb")

In [ ]:
dataset

In [ ]:
dataset.set_format(type="pandas")
dataset["train"][:]

## Tokenizer und Model

In [ ]:
from transformers import TFDistilBertForSequenceClassification, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('distilbert-base-cased')
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-cased', num_labels=2)

In [ ]:
example = "I think I will make a movie next weekend. Oh wait, I'm working..oh I'm sure I can fit it in. It looks like whoever made this film fit it in. I hope the makers of this crap have day jobs because this film sucked!!! It looks like someones home movie and I don't think more than $100 was spent making it!!! Total crap!!! Who let's this stuff be released?!?!?!"
token_ids = tokenizer(example, return_tensors="tf")["input_ids"]
token_ids

In [ ]:
tokenizer.convert_ids_to_tokens(token_ids[0])

In [ ]:
from tensorflow.data import Dataset

train_embeddings = tokenizer(list(dataset["train"]["text"][:1000]), truncation=True, padding=True)
test_embeddings = tokenizer(list(dataset["test"]["text"][:100]), truncation=True, padding=True)

train = Dataset.from_tensor_slices((dict(train_embeddings), dataset["train"]["label"][:1000]))
test = Dataset.from_tensor_slices((dict(test_embeddings), dataset["test"]["label"][:100]))

### Modeling

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy

optimizerr = Adam(learning_rate=5e-5)
losss = SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizerr, loss=losss, metrics=['accuracy'])
model.fit(train.shuffle(1000).batch(16), epochs=1, batch_size=16)

# Evaluation

In [ ]:
loss, acc = model.evaluate(test.batch(16))
acc

### Modeling

---

Wahlpflichtach Künstliche Intelligenz II: Praktikum 